# Deep learning-based object detection with OpenCV MobileNet:SSD

we will use the MobileNet SSD + deep neural network (dnn
 ) module in OpenCV to build our object detector.

Imports Libraries

In [1]:
import cv2
import numpy as np
import argparse

Constrauct Arguments parse and parse the arguments

In [2]:
ap=argparse.ArgumentParser()

ap.add_argument("--image",required=True,help="path of the image")
ap.add_argument("--prototxt",required=True,help="path to Caffe 'deploy' prototxt file")
ap.add_argument("--model",required=True,help="path of caffe 'deploy' pretrained model")
ap.add_argument("--confidence",required=True,help="minimum probabilty to filter weak predictions")

args=vars(ap.parse_args(["--image",r"C:\Users\SRKT\Desktop\MibileNet_SSD_modle\images\example_05.jpg",
                        "--prototxt",r"C:\Users\SRKT\Desktop\MibileNet_SSD_modle\MobileNetSSD_deploy.prototxt.txt",
                        "--model",r"C:\Users\SRKT\Desktop\MibileNet_SSD_modle\MobileNetSSD_deploy.caffemodel",
                        "--confidence","0.5"]))

initialize class labels and bounding box colors:

In [3]:
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
"sofa", "train", "tvmonitor"]

COLORS=np.random.uniform(0,255,size=(len(CLASSES),3))


Load Model

In [4]:
print("[INFO] loading model")

net=cv2.dnn.readNetFromCaffe(args["prototxt"],args["model"])


[INFO] loading model


load our query image and prepare the blob
#load the input image and construct an input blob for the image
#by resizing to a fixed 300x300 pixels and then normalizing it
#(note: normalization is done via the authors of the MobileNet SSD
#implementation)

In [5]:
image=cv2.imread(args["image"])
(h,w)=image.shape[:2]

blob=cv2.dnn.blobFromImage(cv2.resize(image,(300,300)),0.007843,(300, 300), 127.5)


we’ll pass this blob through the neural network:

 pass the blob through the network and obtain the detections and
 predictions

In [6]:
print("[INFO] computing object detections ....")

net.setInput(blob)
detections=net.forward()

[INFO] computing object detections ....


[,frame,no of detections,[class id,class score,conf,x,y,h,w]]

In [7]:
detections.shape

(1, 1, 100, 7)

Let’s loop through our detections
  and determine what and where the objects are in the image:

In [8]:
#Looping through our detections
# , first we extract the confidence
for i in range(0,detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the
    # prediction
    confidence=detections[0,0,i,2]
    
    # filter out weak detections by ensuring the `confidence` is
    # greater than the minimum confidence
    if confidence>0.5:
        
        # extract the index of the class label from the `detections`,
        # then compute the (x, y)-coordinates of the bounding box for
        # the object
        idx=int(detections[0,0,i,1])
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        # we build a text label containing the CLASS name and the confidence
        label="{} : {:.2f}%".format(CLASSES[idx],confidence*100)
        print("[INFO] {}".format(label))
        
        cv2.rectangle(image,(startX,startY),(endX,endY),COLORS[idx],2)
        y=startY-15 if startY-15>15 else startY+15
        cv2.putText(image,label,(startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.5,COLORS[idx],2)
        
    

[INFO] car : 99.43%
[INFO] cat : 57.60%
[INFO] dog : 61.78%
[INFO] horse : 99.91%
[INFO] person : 88.44%


display the result:

In [ ]:
cv2.imshow("output",image)
cv2.waitKey(0)